Wesley Neves

Projeto Final 01 – Peso: 4 pontos

Utilizando majoritariamente as funções map, filter e reduce do próprio Python (vistas na Aula 07), e considerando a lista de salários mensais (em reais) dos funcionários de uma determinada empresa nacional, recém-adquirida por uma empresa multinacional, armazenados no arquivo “salarios.csv” (disponível no próprio Moodle), codifique em Python e responda às seguintes perguntas:

1)    Qual o maior salário (em reais)?

2)    Qual o menor salário (em reais)?

3)    Converta todos os salários (em reais) para dólares e encontre a soma total dos salários dos funcionários da companhia (resultado também em dólares) – considere 1 dólar equivalente a 4 reais.

4)    Encontre o salário médio dos funcionários (em dólares) e liste todos os salários existentes acima deste valor (valores também em dólares).

5)    Se o presidente da multinacional quiser aumentar em 10% todos os salários da empresa abaixo do salário médio, quanto ele gastaria a mais por mês com os funcionários (calcule e responda em dólares)?

OBS: Não é necessário enviar um relatório/texto escrito com as respostas, documente cada linha de código de seu programa e comente as respostas após cada linha de comando, se possível em um Jupyter Notebook. Poste o código bem documentado (com comentários, etc.)  aqui.

Não esqueça de identificar-se.

In [135]:
import numpy as np
import pandas as pd
import functools as fn

from pyspark import SparkContext
sc=SparkContext.getOrCreate()

In [136]:
df = pd.read_csv('salarios.csv')

In [139]:


df.columns =["Salario"]

df["Salario"].astype(float)

df.shape

(19999, 1)

In [140]:
df.head(5)

,Salario
0,19467.0
1,7334.0
2,27500.0
3,20169.0
4,16724.0


In [141]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import *
spark = SparkSession(sc)

In [142]:
data_Spark = spark.createDataFrame(df)

In [143]:
data_Spark

DataFrame[Salario: double]

In [144]:
rddList = data_Spark.rdd

In [145]:
type(rddList)

pyspark.rdd.RDD

## 1) Qual o maior salário (em reais)?

In [146]:
get_maior_salario =  rddList.reduce(lambda x, y : x if(x > y) else y)

print(get_maior_salario)
## em python
# fn.reduce(lambda x, y : x if(x > y) else y,df["Salario"])

Row(Salario=33765.0)


## 2) Qual o menor salário (em reais)?

In [147]:
get_menor_salario =  rddList.reduce(lambda x, y : x if(x < y) else y)


print(get_menor_salario)
## em python 
# fn.reduce(lambda x, y : x if(x < y) else y,df["Salario"])


Row(Salario=1001.0)


In [150]:
rddList.take(10)

[Row(Salario=19467.0),
 Row(Salario=7334.0),
 Row(Salario=27500.0),
 Row(Salario=20169.0),
 Row(Salario=16724.0),
 Row(Salario=12478.0),
 Row(Salario=30358.0),
 Row(Salario=27962.0),
 Row(Salario=25464.0),
 Row(Salario=6705.0)]

## 3) Converta todos os salários (em reais) para dólares e encontre a soma total dos salários dos funcionários da companhia (resultado também em dólares) – considere 1 dólar equivalente a 4 reais.

In [151]:
valor_dolar = 4

def converteRealEmDolar(valor):
    return (valor * valor_dolar)


In [153]:
df = rddList.map(lambda x: (x["Salario"],converteRealEmDolar(x["Salario"]))).toDF(["Salario", "SalarioEmDolar"])

df.take(10)

[Row(Salario=19467.0, SalarioEmDolar=77868.0),
 Row(Salario=7334.0, SalarioEmDolar=29336.0),
 Row(Salario=27500.0, SalarioEmDolar=110000.0),
 Row(Salario=20169.0, SalarioEmDolar=80676.0),
 Row(Salario=16724.0, SalarioEmDolar=66896.0),
 Row(Salario=12478.0, SalarioEmDolar=49912.0),
 Row(Salario=30358.0, SalarioEmDolar=121432.0),
 Row(Salario=27962.0, SalarioEmDolar=111848.0),
 Row(Salario=25464.0, SalarioEmDolar=101856.0),
 Row(Salario=6705.0, SalarioEmDolar=26820.0)]

## recuperar a soma total de salarios em dolar

In [154]:
total = df.rdd.map(lambda s: s["SalarioEmDolar"]).reduce(lambda a, b: a + b)
print(total)

1400973308.0


## 4)    Encontre o salário médio dos funcionários (em dólares) e liste todos os salários existentes acima deste valor (valores também em dólares).

### função de exemplo

### Salario Medio

## liste todos os salários existentes acima deste valor

5)    Se o presidente da multinacional quiser aumentar em 10% todos os salários da empresa abaixo do salário médio, quanto ele gastaria a mais por mês com os funcionários (calcule e responda em dólares)?

### Isola somente os funcionarios que estão abaixo da media

### aplica o aumento em 10 %